In [354]:
# Biblioteki wejściowe
import pandas as pd
import numpy as np
import posixpath as os
import pathlib
from pathlib import Path
import pyodbc
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
#pd.options.display.min_colwidth=10
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

DATA = Path(r'C:\Users\DW\KOPALNIA\MIS_REPORT_2022\MONTHLY_REPORTS_2022\CF_202207\DATA')
ASSETS = Path(r'C:\Users\DW\KOPALNIA\MIS_REPORT_2022\MONTHLY_REPORTS_2022\CF_202207\ASSETS')
IMAGES = Path(r'C:\Users\DW\KOPALNIA\MIS_REPORT_2022\MONTHLY_REPORTS_2022\CF_202207\IMAGES')

msc=7

In [ ]:
# ZAPYTANIE SQL

t_sql="SELECT Konta.Symbol, Konta.Nazwa, ZapisyKsiegowe.NumerDokumentu,ZapisyKsiegowe.Opis,ZapisyKsiegowe.Data, ZapisyKsiegowe.Strona, ZapisyKsiegowe.KwotaZapisuValue FROM ZapisyKsiegowe INNER JOIN Konta ON ZapisyKsiegowe.Konto = Konta.ID AND ZapisyKsiegowe.Okres = Konta.Okres WHERE (ZapisyKsiegowe.Data >= CONVERT(DATETIME, '2022-01-01 00:00:00', 102))"


cnxn=pyodbc.connect('Driver={SQL Server};'
                    'Server=192.168.1.252\ENOVA_KOPALNIA;'
                    'Database=OGORZELEC;'
                    'UID=ro_daniel.wojcik;'
                    'PWD=661594050xkr@;'
                    'Trusted_Connection=no;')

t_zapisy=pd.read_sql_query (t_sql,cnxn)
t_zapisy.to_csv(DATA /'CF_Zapisy_SQL.csv',index=False)

In [164]:
# Pobiera dane wejściowe MAC

t_zapisy=pd.read_csv(DATA /'CF_Zapisy_SQL.csv')


In [165]:
# Pobiera wykonanie z zapisów księgowych

t_zapisy['Data']=pd.to_datetime(t_zapisy['Data'], infer_datetime_format=True)

t_zapisy['Kwota_DT']=t_zapisy.apply(lambda x:x['KwotaZapisuValue'] if x['Strona']!=1 else 0,axis=1)
t_zapisy['Kwota_CR']=t_zapisy.apply(lambda x:x['KwotaZapisuValue'] if x['Strona']==1 else 0,axis=1)
t_zapisy['Kwota_SAL']=t_zapisy['Kwota_DT']-t_zapisy['Kwota_CR']


t_zapisy.drop(['Strona'],axis=1,inplace=True)
t_zapisy.drop(['KwotaZapisuValue'],axis=1,inplace=True)

t_zapisy.rename(columns={"Symbol":'Konto','NumerDokumentu':'N_Dokument'},inplace=True)
t_zapisy['Data_M']=t_zapisy['Data']+MonthEnd(0)
t_zapisy['R']=t_zapisy['Data'].dt.year
t_zapisy['M']=t_zapisy['Data'].dt.month
t_zapisy['ID']=0
t_zapisy=t_zapisy[t_zapisy['Konto'].str.startswith('9')==False]

In [166]:
# t_mapa_sprawozdania

l_mapa=[['Aktywa',
  'A. Aktywa trwałe',
  'I. Wartości niematerialne i prawne',
  '1. Wartości niematerialne i prawne',
  '1. Wartości niematerialne i prawne',
  'A1'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'II. Rzeczowe aktywa trwałe',
  '2. Środki trwałe',
  'a) Grunty (w tym prawo uw)',
  'A2'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'II. Rzeczowe aktywa trwałe',
  '2. Środki trwałe',
  'b) Budynki, lokale etc.',
  'A3'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'II. Rzeczowe aktywa trwałe',
  '2. Środki trwałe',
  'c) Urządzenia techniczne i maszyny',
  'A4'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'II. Rzeczowe aktywa trwałe',
  '2. Środki trwałe',
  'd) Środki transportu',
  'A5'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'II. Rzeczowe aktywa trwałe',
  '2. Środki trwałe',
  'e) Inne środki trwałe',
  'A6'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'II. Rzeczowe aktywa trwałe',
  '3. Środki trwałe w budowie',
  '3. Środki trwałe w budowie',
  'A7'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'III. Należności długoterminowe',
  'III. Należności długoterminowe',
  'III. Należności długoterminowe',
  'A8'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'IV. Inwestycje długoterminowe',
  '1. Długoterminowe aktywa finansowe',
  '1. Długoterminowe aktywa finansowe',
  'A9'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'V. Długoterminowe rozliczenia międzyokresowe',
  '1. Aktywa z tytułu odroczonego podatku dochodowego',
  '1. Aktywa z tytułu odroczonego podatku dochodowego',
  'A10'],
 ['Aktywa',
  'A. Aktywa trwałe',
  'V. Długoterminowe rozliczenia międzyokresowe',
  '2. Inne rozliczenia międzyokresowe',
  '2. Inne rozliczenia międzyokresowe',
  'A11'],
 ['Aktywa',
  'B. Aktywa obrotowe',
  'I. Zapasy',
  '1. Materiały',
  '1. Materiały',
  'A12'],
 ['Aktywa',
  'B. Aktywa obrotowe',
  'I. Zapasy',
  '2. Produkty gotowe',
  '2. Produkty gotowe',
  'A13'],
 ['Aktywa',
  'B. Aktywa obrotowe',
  'II. Należności krótkoterminowe',
  '1. Należności od jednostek powiązanych',
  'a) z tytułu dostaw i usług',
  'A14'],
 ['Aktywa',
  'B. Aktywa obrotowe',
  'II. Należności krótkoterminowe',
  '2. Należności od pozostałych jednostek',
  'a) z tytułu dostaw i usług',
  'A15'],
 ['Aktywa',
  'B. Aktywa obrotowe',
  'II. Należności krótkoterminowe',
  '2. Należności od pozostałych jednostek',
  'b) z tytułu podatków, dotacji, ceł, etc.',
  'A16'],
 ['Aktywa',
  'B. Aktywa obrotowe',
  'II. Należności krótkoterminowe',
  '2. Należności od pozostałych jednostek',
  'c) inne',
  'A17'],
 ['Aktywa',
  'B. Aktywa obrotowe',
  'III. Inwestycje krótkoterminowe',
  '1. Krótkoterminowe aktywa finansowe',
  'a) środki pieniężne i inne aktywa pieniężne',
  'A18'],
 ['Aktywa',
  'B. Aktywa obrotowe',
  'IV. Krótkoterminowe rozliczenia międzyokresowe',
  '1. Krótkoterminowe rozliczenia międzyokresowe',
  '1. Krótkoterminowe rozliczenia międzyokresowe',
  'A19'],
 ['Pasywa',
  'A. Kapitał (fundusz) własny',
  'I. Kapitał (fundusz) podstawowy',
  'I. Kapitał (fundusz) podstawowy',
  'I. Kapitał (fundusz) podstawowy',
  'P1'],
 ['Pasywa',
  'A. Kapitał (fundusz) własny',
  'II. Kapitał (fundusz) zapasowy',
  'II. Kapitał (fundusz) zapasowy',
  'II. Kapitał (fundusz) zapasowy',
  'P2'],
 ['Pasywa',
  'A. Kapitał (fundusz) własny',
  'III. Kapitał (fundusz) z aktualizacji wyceny',
  'III. Kapitał (fundusz) z aktualizacji wyceny',
  'III. Kapitał (fundusz) z aktualizacji wyceny',
  'P3'],
 ['Pasywa',
  'A. Kapitał (fundusz) własny',
  'IV. Pozostałe kapitały (fundusze) rezerwowe',
  'IV. Pozostałe kapitały (fundusze) rezerwowe',
  'IV. Pozostałe kapitały (fundusze) rezerwowe',
  'P4'],
 ['Pasywa',
  'A. Kapitał (fundusz) własny',
  'V. Zysk (strata) netto',
  'V. Zysk (strata) netto',
  'V. Zysk (strata) netto',
  'P5'],
 ['Pasywa',
  'A. Kapitał (fundusz) własny',
  'VI. Odpisy w ciągu roku obrotowego',
  'VI. Odpisy w ciągu roku obrotowego',
  'VI. Odpisy w ciągu roku obrotowego',
  'P6'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'I. Rezerwy na zobowiązania',
  '1. Rezerwa z tytułu odroczonego podatku dochodowego',
  '1. Rezerwa z tytułu odroczonego podatku dochodowego',
  'P7'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'I. Rezerwy na zobowiązania',
  '2. Rezerwa na świadczenia emerytalne i podobne',
  '2. Rezerwa na świadczenia emerytalne i podobne',
  'P8'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'I. Rezerwy na zobowiązania',
  '3. Pozostałe rezerwy',
  '3. Pozostałe rezerwy',
  'P9'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'II. Zobowiązania długoterminowe',
  '1. Wobec pozostałych jednostek',
  'a) kredyty i pożyczki',
  'P10'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'III. Zobowiązania krótkoterminowe',
  '1. Zobowiązania wobec jednostek powiązanych',
  'a) kredyty i pożyczki',
  'P11'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'III. Zobowiązania krótkoterminowe',
  '1. Zobowiązania wobec jednostek powiązanych',
  'b) tytułu dostaw i usług',
  'P12'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'III. Zobowiązania krótkoterminowe',
  '2. Zobowiązania wobec pozostałych jednostek',
  'a) kredyty i pożyczki',
  'P13'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'III. Zobowiązania krótkoterminowe',
  '2. Zobowiązania wobec pozostałych jednostek',
  'b) tytułu dostaw i usług',
  'P14'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'III. Zobowiązania krótkoterminowe',
  '2. Zobowiązania wobec pozostałych jednostek',
  'c) z tytułu podatków, ceł, etc.',
  'P15'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'III. Zobowiązania krótkoterminowe',
  '2. Zobowiązania wobec pozostałych jednostek',
  'd) z tytułu wynagrodzeń',
  'P16'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'III. Zobowiązania krótkoterminowe',
  '2. Zobowiązania wobec pozostałych jednostek',
  'e) inne',
  'P17'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'III. Zobowiązania krótkoterminowe',
  '3. Fundusze specjalne',
  '3. Fundusze specjalne',
  'P18'],
 ['Pasywa',
  'B. Zobowiązania i rezerwy na zobowiązania',
  'IV. Rozliczenia międzyokresowe',
  '1. Inne rozliczenia międzyokresowe',
  '1. Inne rozliczenia międzyokresowe',
  'P19'],
 ['Rachunek Wyników',
  'A. Przychody netto ze sprzedaży i zrównane z nimi',
  '1. Przychody netto ze sprzedaży produktów',
  '1. Przychody netto ze sprzedaży produktów',
  '1. Przychody netto ze sprzedaży produktów',
  'R1'],
 ['Rachunek Wyników',
  'A. Przychody netto ze sprzedaży i zrównane z nimi',
  '2.Przychody netto ze sprzedaży towarów i materiałów',
  '2.Przychody netto ze sprzedaży towarów i materiałów',
  '2.Przychody netto ze sprzedaży towarów i materiałów',
  'R2'],
 ['Rachunek Wyników',
  'A. Przychody netto ze sprzedaży i zrównane z nimi',
  '3. Koszt wytworzenia produktów na własne potrzeby',
  '3. Koszt wytworzenia produktów na własne potrzeby',
  '3. Koszt wytworzenia produktów na własne potrzeby',
  'R3'],
 ['Rachunek Wyników',
  'A. Przychody netto ze sprzedaży i zrównane z nimi',
  '4. Zmiana stanu produktów',
  '4. Zmiana stanu produktów',
  '4. Zmiana stanu produktów',
  'R4'],
 ['Rachunek Wyników',
  'B. Koszty działalności operacyjnej',
  '1. Amortyzacja',
  '1. Amortyzacja',
  '1. Amortyzacja',
  'R5'],
 ['Rachunek Wyników',
  'B. Koszty działalności operacyjnej',
  '2. Zużycie materiałów i energii',
  '2. Zużycie materiałów i energii',
  '2. Zużycie materiałów i energii',
  'R6'],
 ['Rachunek Wyników',
  'B. Koszty działalności operacyjnej',
  '3. Usługi obce',
  '3. Usługi obce',
  '3. Usługi obce',
  'R7'],
 ['Rachunek Wyników',
  'B. Koszty działalności operacyjnej',
  '4. Podatki i opłaty',
  '4. Podatki i opłaty',
  '4. Podatki i opłaty',
  'R8'],
 ['Rachunek Wyników',
  'B. Koszty działalności operacyjnej',
  '5. Wynagrodzenia',
  '5. Wynagrodzenia',
  '5. Wynagrodzenia',
  'R9'],
 ['Rachunek Wyników',
  'B. Koszty działalności operacyjnej',
  '6. Ubezpieczenia społeczne i inne świadczenia, w tym:',
  '6. Ubezpieczenia społeczne i inne świadczenia, w tym:',
  '6. Ubezpieczenia społeczne i inne świadczenia, w tym:',
  'R10'],
 ['Rachunek Wyników',
  'B. Koszty działalności operacyjnej',
  '7. Pozostałe koszty rodzajowe',
  '7. Pozostałe koszty rodzajowe',
  '7. Pozostałe koszty rodzajowe',
  'R11'],
 ['Rachunek Wyników',
  'B. Koszty działalności operacyjnej',
  '8. Wartość sprzedanych towarów i materiałów',
  '8. Wartość sprzedanych towarów i materiałów',
  '8. Wartość sprzedanych towarów i materiałów',
  'R12'],
 ['Rachunek Wyników',
  'C. Pozostałe przychody operacyjne',
  '1. Zysk z tytułu rozchodu niefinansowych aktywów trwałych',
  '1. Zysk z tytułu rozchodu niefinansowych aktywów trwałych',
  '1. Zysk z tytułu rozchodu niefinansowych aktywów trwałych',
  'R13'],
 ['Rachunek Wyników',
  'C. Pozostałe przychody operacyjne',
  '2. Inne przychody operacyjne',
  '2. Inne przychody operacyjne',
  '2. Inne przychody operacyjne',
  'R14'],
 ['Rachunek Wyników',
  'D. Pozostałe koszty operacyjne',
  '1. Inne koszty operacyjne',
  '1. Inne koszty operacyjne',
  '1. Inne koszty operacyjne',
  'R15'],
 ['Rachunek Wyników',
  'E. Przychody finansowe',
  '1. Odsetki',
  '1. Odsetki',
  '1. Odsetki',
  'R16'],
 ['Rachunek Wyników',
  'F. Koszty finansowe',
  '1. Odsetki',
  '1. Odsetki',
  '1. Odsetki',
  'R17'],
 ['Rachunek Wyników',
  'F. Koszty finansowe',
  '2. Inne',
  '2. Inne',
  '2. Inne',
  'R18'],
 ['Rachunek Wyników',
  'G. Podatek dochodowy',
  'G. Podatek dochodowy',
  'G. Podatek dochodowy',
  'G. Podatek dochodowy',
  'R19']]

In [167]:
t_mapa=pd.DataFrame(l_mapa,columns=['P0','P1','P2','P3','P4','ID'])

## Bilans - aktywa

In [220]:
# WNIP

t_zapisy['ID']=t_zapisy.apply(lambda x:'A1' if (
    (x['Konto'].startswith('020'))
    or (x['Konto'].startswith('072'))
    )
else x['ID'],axis=1)

In [221]:
# Grunty

t_zapisy['ID']=t_zapisy.apply(lambda x:'A2' if (
    (x['Konto'].startswith('010-001'))
    or (x['Konto'].startswith('010-002'))
    or (x['Konto'].startswith('010-003'))
    or (x['Konto'].startswith('071-000'))
    or (x['Konto'].startswith('071-002'))
    )
else x['ID'],axis=1)

In [222]:
# Budynki

t_zapisy['ID']=t_zapisy.apply(lambda x:'A3' if (
    (x['Konto'].startswith('010-100'))
    or (x['Konto'].startswith('010-200'))
    or (x['Konto'].startswith('071-100'))
    or (x['Konto'].startswith('071-200'))
    )
else x['ID'],axis=1)

In [223]:
# Urządzenia techniczne i maszyny

t_zapisy['ID']=t_zapisy.apply(lambda x:'A4' if (
    (x['Konto'].startswith('010-400'))
    or (x['Konto'].startswith('010-500'))
    or (x['Konto'].startswith('010-600'))
    or (x['Konto'].startswith('071-400'))
    or (x['Konto'].startswith('071-500'))
    or (x['Konto'].startswith('071-600'))
    )
else x['ID'],axis=1)

In [224]:
# Środki transportu

t_zapisy['ID']=t_zapisy.apply(lambda x:'A5' if (
    (x['Konto'].startswith('010-700'))
    or (x['Konto'].startswith('071-700'))
    )
else x['ID'],axis=1)

In [225]:
# Inne środki trwałe

t_zapisy['ID']=t_zapisy.apply(lambda x:'A6' if (
    (x['Konto'].startswith('010-800'))
    or (x['Konto'].startswith('071-800'))
    )
else x['ID'],axis=1)

In [226]:
# Środki trwałe w budowie

t_zapisy['ID']=t_zapisy.apply(lambda x:'A7' if (
    (x['Konto'].startswith('080'))
    )
else x['ID'],axis=1)

In [227]:
# Aktywa z odroczonego podatku dochodowego

t_zapisy['ID']=t_zapisy.apply(lambda x:'A10' if (
    (x['Konto'].startswith('650-001'))
    )
else x['ID'],axis=1)

In [228]:
# Inne rozliczenia międzyokresowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'A11' if (
    (x['Konto'].startswith('645'))
    )
else x['ID'],axis=1)

In [229]:
# Zapasy - materiały

t_zapisy['ID']=t_zapisy.apply(lambda x:'A12' if (
    (x['Konto'].startswith('311'))
    or (x['Konto'].startswith('300'))
    )
else x['ID'],axis=1)

In [230]:
# Zapasy - produkty gotowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'A13' if (
    (x['Konto'].startswith('600'))
    )
else x['ID'],axis=1)

In [231]:
# Należności od pozostałych jednostek - z tytułu dostaw i usług

t_zapisy['ID']=t_zapisy.apply(lambda x:'A15' if (
    (x['Konto'].startswith('200'))
    or (x['Konto'].startswith('280'))
    )
else x['ID'],axis=1)

In [232]:
# Należności z tytułu podatków

t_zapisy['ID']=t_zapisy.apply(lambda x:'A16' if (
    (x['Konto'].startswith('220')) # tylko strona WN
    or (x['Konto'].startswith('221'))
    or (x['Konto'].startswith('222'))
    )
else x['ID'],axis=1)

In [233]:
# Należności inne

t_zapisy['ID']=t_zapisy.apply(lambda x:'A17' if (
    (x['Konto'].startswith('234'))
    #or (x['Konto'].startswith('220')) # tylko strona MA
    or (x['Konto'].startswith('249-100-01'))
    or (x['Konto'].startswith('249-100-09'))
    )
else x['ID'],axis=1)

In [234]:
# Środki pieniężne w kasie i na rachunkach

t_zapisy['ID']=t_zapisy.apply(lambda x:'A18' if (
    (x['Konto'].startswith('100'))
    or (x['Konto'].startswith('130'))
    or (x['Konto'].startswith('131'))
    or (x['Konto'].startswith('140'))
    )
else x['ID'],axis=1)

In [235]:
# Krótkoterminowe rozliczenia RMK

t_zapisy['ID']=t_zapisy.apply(lambda x:'A19' if (
    (x['Konto'].startswith('640'))
    or (x['Konto'].startswith('641'))
    or (x['Konto'].startswith('646'))
    )
else x['ID'],axis=1)

## Bilans - Pasywa

In [236]:
# Kapitał podstawowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P1' if (
    (x['Konto'].startswith('800'))
    )
else x['ID'],axis=1)

In [237]:
# Kapitał zapasowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P2' if (
    (x['Konto'].startswith('811'))
    )
else x['ID'],axis=1)

In [238]:
# Kapitał z aktualizacji

t_zapisy['ID']=t_zapisy.apply(lambda x:'P3' if (
    (x['Konto'].startswith('814'))
    )
else x['ID'],axis=1)

In [239]:
# Kapitał rezerwowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P4' if (
    (x['Konto'].startswith('812'))
    )
else x['ID'],axis=1)

In [266]:
# Kapitał rezerwowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P5' if (
    (x['Konto'].startswith('860'))
    )
else x['ID'],axis=1)

In [240]:
# Odpisy z zysku w ciągu roku

t_zapisy['ID']=t_zapisy.apply(lambda x:'P6' if (
    (x['Konto'].startswith('820'))
    )
else x['ID'],axis=1)

In [241]:
# Rezerwa z tytułu odroczonego podatku dochodowego

t_zapisy['ID']=t_zapisy.apply(lambda x:'P7' if (
    (x['Konto'].startswith('830'))
    )
else x['ID'],axis=1)

In [242]:
# Rezerwa na świadczenia emerytalne

t_zapisy['ID']=t_zapisy.apply(lambda x:'P8' if (
    (x['Konto'].startswith('642-002'))
    or (x['Konto'].startswith('642-005'))
    or (x['Konto'].startswith('642-001'))
    or (x['Konto'].startswith('642-003'))
    or (x['Konto'].startswith('642-004'))
    or (x['Konto'].startswith('642-006'))
    )
else x['ID'],axis=1)

In [243]:
# Pozostałe rezerwy

t_zapisy['ID']=t_zapisy.apply(lambda x:'P9' if (
    (x['Konto'].startswith('643-001-02'))
    or (x['Konto'].startswith('640-015'))
    or (x['Konto'].startswith('830-200'))
    or (x['Konto'].startswith('643-001-01'))
    )
else x['ID'],axis=1)

In [244]:
# Kredyt i pożyczki długoterminowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'P10' if (
    (x['Konto'].startswith('135'))
    or (x['Konto'].startswith('241'))
    )
else x['ID'],axis=1)

In [245]:
# Kredyt i pożyczki krótkoterminowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'P13' if (
    (x['Konto'].startswith('134'))
    or (x['Konto'].startswith('240'))
    )
else x['ID'],axis=1)

In [246]:
# Zobowiązania krótkoterminowe - dostawy i usługi

t_zapisy['ID']=t_zapisy.apply(lambda x:'P14' if (
    (x['Konto'].startswith('210'))
    )
else x['ID'],axis=1)

In [247]:
# Zobowiązania krótkoterminowe - podatki

t_zapisy['ID']=t_zapisy.apply(lambda x:'P15' if (
    (x['Konto'].startswith('223-100'))
    or (x['Konto'].startswith('223-200'))
    or (x['Konto'].startswith('223-300'))
    or (x['Konto'].startswith('223-400'))
    or (x['Konto'].startswith('219'))
    )
else x['ID'],axis=1)

In [248]:
# Zobowiązania krótkoterminowe - wynagrodzenia

t_zapisy['ID']=t_zapisy.apply(lambda x:'P16' if (
    (x['Konto'].startswith('230'))
    )
else x['ID'],axis=1)

In [249]:
# Zobowiązania krótkoterminowe - inne

t_zapisy['ID']=t_zapisy.apply(lambda x:'P17' if (
    (x['Konto'].startswith('249'))
    )
else x['ID'],axis=1)

In [250]:
# Fundusze specjalne

t_zapisy['ID']=t_zapisy.apply(lambda x:'P18' if (
    (x['Konto'].startswith('850'))
    or (x['Konto'].startswith('851'))
    )
else x['ID'],axis=1)

In [251]:
# Rozliczenia międzyokresowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'P19' if (
    (x['Konto'].startswith('840-100'))
    )
else x['ID'],axis=1)

## Rachunek wyników

In [252]:
# Przychody ze sprzedaży

t_zapisy['ID']=t_zapisy.apply(lambda x:'R1' if (
    (x['Konto'].startswith('700'))
    or (x['Konto'].startswith('703'))
    )
else x['ID'],axis=1)

In [253]:
# Zmiana stanu produktów

t_zapisy['ID']=t_zapisy.apply(lambda x:'R4' if (
    (x['Konto'].startswith('701-100'))
    or (x['Konto'].startswith('490-02'))
    or (x['Konto'].startswith('791-100'))
    )
else x['ID'],axis=1)

In [254]:
# Koszt wytworzenia produktów na własne potrzeby

t_zapisy['ID']=t_zapisy.apply(lambda x:'R3' if (
    (x['Konto'].startswith('790')) 
    )
else x['ID'],axis=1)

In [255]:
# Przychody ze sprzedaży towarów i materiałów

t_zapisy['ID']=t_zapisy.apply(lambda x:'R2' if (
    (x['Konto'].startswith('730'))
    or (x['Konto'].startswith('740'))
    )
else x['ID'],axis=1)

In [256]:
# Koszty rodzajowe

t_zapisy['ID']=t_zapisy.apply(lambda x:'R5' if ((x['Konto'].startswith('400'))) else x['ID'],axis=1) # amortyzacja
t_zapisy['ID']=t_zapisy.apply(lambda x:'R6' if ((x['Konto'].startswith('401'))) else x['ID'],axis=1) # zużycie materiałów i energii
t_zapisy['ID']=t_zapisy.apply(lambda x:'R7' if ((x['Konto'].startswith('402'))) else x['ID'],axis=1) # usługi obce
t_zapisy['ID']=t_zapisy.apply(lambda x:'R8' if ((x['Konto'].startswith('403'))) else x['ID'],axis=1) # podatki i opłaty
t_zapisy['ID']=t_zapisy.apply(lambda x:'R9' if ((x['Konto'].startswith('404'))) else x['ID'],axis=1) # Wynagrodzenia
t_zapisy['ID']=t_zapisy.apply(lambda x:'R10' if ((x['Konto'].startswith('405'))) else x['ID'],axis=1) # ZUS
t_zapisy['ID']=t_zapisy.apply(lambda x:'R11' if ((x['Konto'].startswith('409'))) else x['ID'],axis=1) # pozostałe

In [257]:
# Wartość sprzedanych towarów i materiałów

t_zapisy['ID']=t_zapisy.apply(lambda x:'R12' if (
    (x['Konto'].startswith('731'))
    or (x['Konto'].startswith('741'))
    )
else x['ID'],axis=1)

In [258]:
# Zysk ze sprzedaży MT

t_zapisy['ID']=t_zapisy.apply(lambda x:'R13' if (
    (x['Konto'].startswith('760-002'))
    or (x['Konto'].startswith('761-011'))
    )
else x['ID'],axis=1)

In [267]:
# Inne przychody operacyjne

t_zapisy['ID']=t_zapisy.apply(lambda x:'R14' if (
    (x['Konto'].startswith('760-001'))
    or (x['Konto'].startswith('760-003'))
    or (x['Konto'].startswith('760-013'))
    or (x['Konto'].startswith('760-005'))
    or (x['Konto'].startswith('761-013'))
    or (x['Konto'].startswith('760-018'))
    or (x['Konto'].startswith('760-006'))
    or (x['Konto'].startswith('760-015'))
    or (x['Konto'].startswith('750-001-02'))
    or (x['Konto'].startswith('760-004'))
    or (x['Konto'].startswith('760-019'))

    )
else x['ID'],axis=1)

In [268]:
# Inne koszty operacyjne

t_zapisy['ID']=t_zapisy.apply(lambda x:'R15' if (
    (x['Konto'].startswith('761-001'))
    or (x['Konto'].startswith('761-010'))
    or (x['Konto'].startswith('761-012'))
    or (x['Konto'].startswith('761-014'))
    or (x['Konto'].startswith('761-022'))
    or (x['Konto'].startswith('761-023'))
    or (x['Konto'].startswith('761-024'))
    or (x['Konto'].startswith('760-007'))
    or (x['Konto'].startswith('761-009-03'))
    or (x['Konto'].startswith('761-009-04'))
    )
else x['ID'],axis=1)

In [261]:
# Koszty finansowe - odsetki

t_zapisy['ID']=t_zapisy.apply(lambda x:'R17' if (
    (x['Konto'].startswith('751-001'))
    or (x['Konto'].startswith('751-003'))
    )
else x['ID'],axis=1)

In [262]:
# Koszty finansowe - inne

t_zapisy['ID']=t_zapisy.apply(lambda x:'R18' if (
    (x['Konto'].startswith('751-002'))
    or (x['Konto'].startswith('751-004'))
    )
else x['ID'],axis=1)

In [263]:
# Podatek dochodowy

t_zapisy['ID']=t_zapisy.apply(lambda x:'R19' if (
    (x['Konto'].startswith('870'))
    )
else x['ID'],axis=1)

In [270]:
# Obrotówka
t_balance=t_zapisy.groupby(['Konto','Nazwa','M','ID'],as_index=False)[['Kwota_DT','Kwota_CR','Kwota_SAL']].sum()
t_balance[t_balance['ID']==0]

,Konto,Nazwa,M,ID,Kwota_DT,Kwota_CR,Kwota_SAL


## Raport

In [383]:
# Buduje raport

t_okres=pd.DataFrame({'M':range(1,13)})
t_konta=t_balance[['Konto','Nazwa','ID']]
temp=t_balance[['Konto','ID','M','Kwota_DT','Kwota_CR','Kwota_SAL']].copy()

t_tech=t_mapa[['ID']].copy()
t_tech['Konto']='000-000'
t_tech['Nazwa']='Zapis techniczny'
t_tech=t_tech[['Konto','Nazwa','ID']]



t_konta=pd.concat([t_konta,t_tech],ignore_index=True)

t_konta=t_konta.drop_duplicates(['Konto','Nazwa','ID'])
t_konta['Konto_N']=t_konta['Konto']+ ' - '+t_konta['Nazwa']
t_konta=pd.merge(t_konta,t_okres,how='cross')
t_konta=pd.merge(t_konta,temp,how='left',on=['Konto','ID','M'])

t_wynik=t_balance[t_balance['ID'].str.startswith('R')].groupby(['M'],as_index=False)[['Kwota_DT','Kwota_CR','Kwota_SAL']].sum()
t_wynik['Konto']='000-000'
t_wynik['Nazwa']='Wynik finansowy'
t_wynik['Konto_N']='000-000 - Wynik finansowy'
t_wynik['ID']='P5'

t_konta=pd.concat([t_konta,t_wynik],ignore_index=True)

t_konta.fillna(0,inplace=True)



t_konta['Kwota_YTD'] = t_konta.groupby('Konto')['Kwota_SAL'].transform(pd.Series.cumsum)
t_konta['Kwota_YTD']=t_konta.apply(lambda x:-x['Kwota_YTD'] if x['ID'].startswith('A') else x['Kwota_YTD'],axis=1)
t_konta['Kwota_SAL']=t_konta.apply(lambda x:-x['Kwota_SAL'] if x['ID'].startswith('A') else x['Kwota_SAL'],axis=1)
t_konta=pd.merge(t_konta,t_mapa,how='left',on=['ID'])


In [386]:
# Sprawozdania

t_tabela=pd.pivot_table(t_konta,index=['P0'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=['Sprawozdanie Fin.',1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,Sprawozdanie Fin.,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,Aktywa,"24,080,803.25","810,545.42","2,440,594.10","1,449,515.51","-553,129.63","-42,511.80","-98,709.87","-2,094,890.66",0,0,0,0,"25,992,216.32"
1,Pasywa,"24,080,803.25","810,545.42","2,440,594.10","1,449,515.51","-553,129.63","-42,511.80","-98,709.87","-2,094,890.66",0,0,0,0,"25,992,216.32"
2,Rachunek Wyników,"-833,041.48","-317,935.25","279,280.61","587,166.61","259,054.94","162,217.72","315,284.98","-118,555.65",0,0,0,0,"333,472.48"
3,RAZEM,"47,328,565.02","1,303,155.59","5,160,468.81","3,486,197.63","-847,204.32","77,194.12","117,865.24","-4,308,336.97",0,0,0,0,"52,317,905.12"


In [387]:
# Tabela Bilansowa

P0='Aktywa'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
],index=['P1'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P0,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela


,Aktywa,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,A. Aktywa trwałe,"17,369,051.92","-123,017.79","-93,157.19","13,193.81","-134,989.21","-177,442.83","64,464.38",0.00,0,0,0,0,"16,918,103.09"
1,B. Aktywa obrotowe,"6,711,751.33","933,563.21","2,533,751.29","1,436,321.70","-418,140.42","134,931.03","-163,174.25","-2,094,890.66",0,0,0,0,"9,074,113.23"
2,RAZEM,"24,080,803.25","810,545.42","2,440,594.10","1,449,515.51","-553,129.63","-42,511.80","-98,709.87","-2,094,890.66",0,0,0,0,"25,992,216.32"


In [361]:
# Tabela Bilansowa

P0='Aktywa'
P1='A. Aktywa trwałe'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
],index=['P2'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P1,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela


,A. Aktywa trwałe,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,I. Wartości niematerialne i prawne,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,-0.00
1,II. Rzeczowe aktywa trwałe,"13,311,584.08","-129,517.79","-93,157.19","13,193.81","-134,989.21","-177,442.83","64,464.38",0,0,0,0,0,"12,854,135.25"
2,III. Należności długoterminowe,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00
3,IV. Inwestycje długoterminowe,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00
4,V. Długoterminowe rozliczenia międzyokresowe,"4,057,467.84","6,500.00",0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,"4,063,967.84"
5,RAZEM,"17,369,051.92","-123,017.79","-93,157.19","13,193.81","-134,989.21","-177,442.83","64,464.38",0,0,0,0,0,"16,918,103.09"


In [364]:
# Tabela Bilansowa

P0='Aktywa'
P1='A. Aktywa trwałe'
P2='I. Wartości niematerialne i prawne'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
],index=['P3'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P2,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,I. Wartości niematerialne i prawne,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,1. Wartości niematerialne i prawne,0,0,0,0,0,0,0,0,0,0,0,0,-0.00
1,RAZEM,0,0,0,0,0,0,0,0,0,0,0,0,-0.00


In [366]:
# Tabela Bilansowa

P0='Aktywa'
P1='A. Aktywa trwałe'
P2='I. Wartości niematerialne i prawne'
P3='1. Wartości niematerialne i prawne'
P4='1. Wartości niematerialne i prawne'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
],index=['Konto_N'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P3,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,1. Wartości niematerialne i prawne,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,000-000 - Zapis techniczny,0.00,0,0,0,0,0,0,0,0,0,0,0,0.00
1,020-004 - WNIP-SYSTEM TU ZARZĄDZANIA PRODUKCJĄ,"61,848.99",0,0,0,0,0,0,0,0,0,0,0,"61,848.99"
2,020-006 - WNIP-KONCESJA CHEŁMCZYK,"3,450.00",0,0,0,0,0,0,0,0,0,0,0,"3,450.00"
3,020-008 - WNIP-PAKIET ENOVA,"4,035.00",0,0,0,0,0,0,0,0,0,0,0,"4,035.00"
4,020-009 - WNIP-PR.FIN.KS.DYNAMICS NAV,"83,861.40",0,0,0,0,0,0,0,0,0,0,0,"83,861.40"
5,020-012 - WNIP-KONCESJA OGORZELEC I,"88,616.00",0,0,0,0,0,0,0,0,0,0,0,"88,616.00"
6,020-013 - MICROSOFT.DYN.NAV-FUNKCJON.CE,"6,950.00",0,0,0,0,0,0,0,0,0,0,0,"6,950.00"
7,020-014 - WNIP-PROGRAM WAGOWY WAGMASTER,"13,200.00",0,0,0,0,0,0,0,0,0,0,0,"13,200.00"
8,020-015 - SYSTEM WAGO-MONITOR,"9,500.00",0,0,0,0,0,0,0,0,0,0,0,"9,500.00"
9,020-016 - SYSTEM ENOVA,"24,842.05",0,0,0,0,0,0,0,0,0,0,0,"24,842.05"


In [359]:
# Tabela Bilansowa

P0='Aktywa'
P1='A. Aktywa trwałe'
P2='II. Rzeczowe aktywa trwałe'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
],index=['P3'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P2,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,II. Rzeczowe aktywa trwałe,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,2. Środki trwałe,"12,455,348.82","-129,517.79","-94,757.19","8,846.81","-129,989.21","-177,442.83","-137,054.62",0,0,0,0,0,"11,795,433.99"
1,3. Środki trwałe w budowie,"856,235.26",0.00,"1,600.00","4,347.00","-5,000.00",0.00,"201,519.00",0,0,0,0,0,"1,058,701.26"
2,RAZEM,"13,311,584.08","-129,517.79","-93,157.19","13,193.81","-134,989.21","-177,442.83","64,464.38",0,0,0,0,0,"12,854,135.25"


In [357]:
# Tabela Bilansowa

P0='Aktywa'
P1='A. Aktywa trwałe'
P2='II. Rzeczowe aktywa trwałe'
P3='2. Środki trwałe'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
],index=['P4'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P3,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,2. Środki trwałe,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,a) Grunty (w tym prawo uw),"292,819.57","-1,449.93","-1,449.93","-1,449.93","-1,449.93","-1,449.93","-1,449.93",0,0,0,0,0,"284,119.99"
1,"b) Budynki, lokale etc.","2,684,132.03","-14,271.63","1,868.97","-14,213.97","-14,213.97","-14,213.97","-14,213.97",0,0,0,0,0,"2,614,873.49"
2,c) Urządzenia techniczne i maszyny,"8,672,274.29","-98,818.83","-80,198.83","-98,712.91","-99,347.91","-91,083.55","-106,088.29",0,0,0,0,0,"8,098,023.97"
3,d) Środki transportu,"640,640.39","-12,172.48","-12,172.48","126,028.54","-12,172.48","-67,890.46","-12,497.51",0,0,0,0,0,"649,763.52"
4,e) Inne środki trwałe,"165,482.54","-2,804.92","-2,804.92","-2,804.92","-2,804.92","-2,804.92","-2,804.92",0,0,0,0,0,"148,653.02"
5,RAZEM,"12,455,348.82","-129,517.79","-94,757.19","8,846.81","-129,989.21","-177,442.83","-137,054.62",0,0,0,0,0,"11,795,433.99"


In [355]:
# Tabela Bilansowa

P0='Aktywa'
P1='A. Aktywa trwałe'
P2='II. Rzeczowe aktywa trwałe'
P3='2. Środki trwałe'
P4='a) Grunty (w tym prawo uw)'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
    &(t_konta['P4']==P4)
],index=['Konto_N'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P4,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,a) Grunty (w tym prawo uw),1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,000-000 - Zapis techniczny,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00
1,010-001 - PR.WIECZ.UŻYTKOWANIA GRUNTÓW,"113,234.57",0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,"113,234.57"
2,010-002 - PR.W.UŻ.GRUNTÓW-WIEŚCISZOWICE,"347,982.00",0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,"347,982.00"
3,010-003 - GRUNTY WŁASNE,"211,624.62",0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,"211,624.62"
4,071-000 - UM.WIECZYSTE UŻ.GRUNTÓW,"-113,234.57",0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,"-113,234.57"
5,071-002 - UM.WIECZYSTE UŻ.GR.WIEŚCISZOWICE,"-266,787.05","-1,449.93","-1,449.93","-1,449.93","-1,449.93","-1,449.93","-1,449.93",0,0,0,0,0,"-275,486.63"
6,RAZEM,"292,819.57","-1,449.93","-1,449.93","-1,449.93","-1,449.93","-1,449.93","-1,449.93",0,0,0,0,0,"284,119.99"


In [349]:
# Aktywa trwałe - środki trwałe - budynki
P0='Aktywa'
P1='A. Aktywa trwałe'
P2='II. Rzeczowe aktywa trwałe'
P3='2. Środki trwałe'
P4='b) Budynki, lokale etc.'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
    &(t_konta['P4']==P4)
],index=['Konto_N'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P4,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,"b) Budynki, lokale etc.",1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,000-000 - Zapis techniczny,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00
1,010-100 - BUDYNKI,"375,806.90",0.00,"16,117.28",0.00,0.00,0.00,0.00,0,0,0,0,0,"391,924.18"
2,010-200 - BUDOWLE-OBIEKTY INŻYNIERII,"3,882,908.57",0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,"3,882,908.57"
3,071-100 - UM. BUDYNKÓW,"-140,314.56",-772.00,-748.68,-714.34,-714.34,-714.34,-714.34,0,0,0,0,0,"-144,692.60"
4,071-200 - UM.BUDOWLI-OB.INŻYNIERII,"-1,434,268.88","-13,499.63","-13,499.63","-13,499.63","-13,499.63","-13,499.63","-13,499.63",0,0,0,0,0,"-1,515,266.66"
5,RAZEM,"2,684,132.03","-14,271.63","1,868.97","-14,213.97","-14,213.97","-14,213.97","-14,213.97",0,0,0,0,0,"2,614,873.49"


In [350]:
# Aktywa trwałe - środki trwałe - budynki
P0='Aktywa'
P1='A. Aktywa trwałe'
P2='II. Rzeczowe aktywa trwałe'
P3='2. Środki trwałe'
P4='c) Urządzenia techniczne i maszyny'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
    &(t_konta['P4']==P4)
],index=['Konto_N'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P4,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,c) Urządzenia techniczne i maszyny,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,000-000 - Zapis techniczny,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00
1,010-400 - MASZYNY I URZĄDZ.OGÓLNEGO ZAST.,"729,269.80",0.00,"18,620.00","-4,795.00","-5,079.65",0.00,0.00,0,0,0,0,0,"738,015.15"
2,010-500 - MASZYNY I URZĄDZ.SPECJALNE,"8,306,171.17",0.00,0.00,0.00,0.00,"7,502.37",0.00,0,0,0,0,0,"8,313,673.54"
3,010-600 - URZĄDZENIA TECHNICZNE,"6,683,569.93",0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,"6,683,569.93"
4,071-400 - UM.MASZYN I URZĄDZ.OGÓLNEGO ZAST.,"-407,636.72","-6,703.04","-6,703.04","-1,802.12","-2,152.47","-6,470.13","-6,470.13",0,0,0,0,0,"-437,937.65"
5,071-500 - UM.MASZYN SPECJALNYCH,"-4,194,435.80","-50,739.20","-50,739.20","-50,739.20","-50,739.20","-50,739.20","-58,241.57",0,0,0,0,0,"-4,506,373.37"
6,071-600 - UM.URZĄDZEŃ TECHNICZNYCH,"-2,444,664.09","-41,376.59","-41,376.59","-41,376.59","-41,376.59","-41,376.59","-41,376.59",0,0,0,0,0,"-2,692,923.63"
7,RAZEM,"8,672,274.29","-98,818.83","-80,198.83","-98,712.91","-99,347.91","-91,083.55","-106,088.29",0,0,0,0,0,"8,098,023.97"


In [351]:
# Aktywa trwałe - środki trwałe - budynki
P0='Aktywa'
P1='A. Aktywa trwałe'
P2='II. Rzeczowe aktywa trwałe'
P3='2. Środki trwałe'
P4='d) Środki transportu'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
    &(t_konta['P4']==P4)
],index=['Konto_N'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P4,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,d) Środki transportu,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,000-000 - Zapis techniczny,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00
1,010-700 - ŚRODKI TRANSPORTOWE,"840,847.97",0.00,0.00,"138,201.02",0.00,"-118,699.19",0.00,0,0,0,0,0,"860,349.80"
2,071-700 - UM.ŚRODKÓW TRANSPORTOWYCH,"-200,207.58","-12,172.48","-12,172.48","-12,172.48","-12,172.48","50,808.73","-12,497.51",0,0,0,0,0,"-210,586.28"
3,RAZEM,"640,640.39","-12,172.48","-12,172.48","126,028.54","-12,172.48","-67,890.46","-12,497.51",0,0,0,0,0,"649,763.52"


In [370]:
# Tabela Bilansowa

P0='Aktywa'
P1='A. Aktywa trwałe'
P2='II. Rzeczowe aktywa trwałe'
P3='3. Środki trwałe w budowie'
P4='3. Środki trwałe w budowie'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
    &(t_konta['P4']==P4)
],index=['Konto_N'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P4,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,3. Środki trwałe w budowie,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,000-000 - Zapis techniczny,0.00,0,0,0,0,0,0,0,0,0,0,0,0.00
1,080-100-001 - ŚT.W BUDOWIE-ZAKUP,0.00,0,0,0,0,0,0,0,0,0,0,0,0.00
2,080-200-003 - ŚT.W BUD. SYSTEM OBIEGU DOKUMENTÓW SALDEO SMART,0.00,0,1600,3400,-5000,0,0,0,0,0,0,0,0.00
3,080-300-001 - ST.W BUD.MODERNIZACJA WK3,"102,000.00",0,0,0,0,0,0,0,0,0,0,0,"102,000.00"
4,080-300-004 - PRZEBUDOWA BUDYNKU ADMINISTRACYJNEGO,"325,436.03",0,0,0,0,0,0,0,0,0,0,0,"325,436.03"
5,080-300-006 - ŚT.W BUD.KONTENER STERÓWKA,"19,899.99",0,0,0,0,0,0,0,0,0,0,0,"19,899.99"
6,080-300-007 - ŚT.W BUD. PRZEBUDOWA ZAKŁADU PRZERÓBCZEGO,"134,973.81",0,0,0,0,0,0,0,0,0,0,0,"134,973.81"
7,080-300-018 - ŚT.W BUD. STUDNIA GŁĘBINOWA,"260,353.33",0,0,0,0,0,0,0,0,0,0,0,"260,353.33"
8,080-300-031 - ŚT.W BUD. CENTRALNE SMAROWANIE,0.00,0,0,0,0,0,0,0,0,0,0,0,0.00
9,080-300-034 - ST.W BUD.KONTENER MAGAZYN ODPADÓW,0.00,0,0,0,0,0,0,0,0,0,0,0,0.00


In [388]:
# Tabela Bilansowa

P0='Aktywa'
P1='B. Aktywa obrotowe'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
],index=['P2'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P1,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,B. Aktywa obrotowe,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,I. Zapasy,"1,559,845.18","31,869.52","66,391.03","-27,288.82","-404,152.38","80,698.21","261,749.11",0.00,0,0,0,0,"1,569,111.85"
1,II. Należności krótkoterminowe,"1,789,138.94","1,779,453.16","2,938,385.34","1,297,888.09","-1,151,080.96","410,585.94","-46,479.43","-4,200,751.49",0,0,0,0,"2,817,139.59"
2,III. Inwestycje krótkoterminowe,"3,050,403.56","-717,237.69","-834,406.37","35,777.22","695,562.71","-669,232.31","-263,331.00","2,105,860.83",0,0,0,0,"3,403,396.95"
3,IV. Krótkoterminowe rozliczenia międzyokresowe,"312,363.65","-160,521.78","363,381.29","129,945.21","441,530.21","312,879.19","-115,112.93",0.00,0,0,0,0,"1,284,464.84"
4,RAZEM,"6,711,751.33","933,563.21","2,533,751.29","1,436,321.70","-418,140.42","134,931.03","-163,174.25","-2,094,890.66",0,0,0,0,"9,074,113.23"


In [389]:
# Tabela Bilansowa

P0='Aktywa'
P1='B. Aktywa obrotowe'
P2='I. Zapasy'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
],index=['P3'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P2,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,I. Zapasy,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,1. Materiały,"197,020.85","87,361.24","-52,018.95","67,275.89","-51,558.66","-25,429.45","11,606.21",0,0,0,0,0,"234,257.13"
1,2. Produkty gotowe,"1,362,824.33","-55,491.72","118,409.98","-94,564.71","-352,593.72","106,127.66","250,142.90",0,0,0,0,0,"1,334,854.72"
2,RAZEM,"1,559,845.18","31,869.52","66,391.03","-27,288.82","-404,152.38","80,698.21","261,749.11",0,0,0,0,0,"1,569,111.85"


In [390]:
# Tabela Bilansowa

P0='Aktywa'
P1='B. Aktywa obrotowe'
P2='I. Zapasy'
P3='1. Materiały'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
],index=['P4'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P3,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,1. Materiały,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,1. Materiały,"197,020.85","87,361.24","-52,018.95","67,275.89","-51,558.66","-25,429.45","11,606.21",0,0,0,0,0,"234,257.13"
1,RAZEM,"197,020.85","87,361.24","-52,018.95","67,275.89","-51,558.66","-25,429.45","11,606.21",0,0,0,0,0,"234,257.13"


In [391]:
# Tabela Bilansowa

P0='Aktywa'
P1='B. Aktywa obrotowe'
P2='I. Zapasy'
P3='1. Materiały'
P4='1. Materiały'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
    &(t_konta['P4']==P4)
],index=['Konto_N'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P4,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,1. Materiały,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,000-000 - Zapis techniczny,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00
1,300-301 - ZAKUP MATERIAŁÓW,"103,833.90",0.00,0.00,"-1,833.90",0.00,0.00,0.00,0,0,0,0,0,"102,000.00"
2,300-303 - ZAKUP USŁUG,0.00,"-5,000.00","-129,685.50","134,685.50",0.00,0.00,0.00,0,0,0,0,0,0.00
3,311-001 - MATERIAŁY W MAGAZYNIE,"93,186.95","92,361.24","77,666.55","-65,575.71","-51,558.66","-25,429.45","11,606.21",0,0,0,0,0,"132,257.13"
4,RAZEM,"197,020.85","87,361.24","-52,018.95","67,275.89","-51,558.66","-25,429.45","11,606.21",0,0,0,0,0,"234,257.13"


In [392]:
# Tabela Bilansowa

P0='Aktywa'
P1='B. Aktywa obrotowe'
P2='I. Zapasy'
P3='2. Produkty gotowe'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
],index=['P4'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P3,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,2. Produkty gotowe,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,2. Produkty gotowe,"1,362,824.33","-55,491.72","118,409.98","-94,564.71","-352,593.72","106,127.66","250,142.90",0,0,0,0,0,"1,334,854.72"
1,RAZEM,"1,362,824.33","-55,491.72","118,409.98","-94,564.71","-352,593.72","106,127.66","250,142.90",0,0,0,0,0,"1,334,854.72"


In [393]:
# Tabela Bilansowa

P0='Aktywa'
P1='B. Aktywa obrotowe'
P2='I. Zapasy'
P3='2. Produkty gotowe'
P4='2. Produkty gotowe'


t_tabela=pd.pivot_table(t_konta[(
    t_konta['P0']==P0)
    &(t_konta['P1']==P1)
    &(t_konta['P2']==P2)
    &(t_konta['P3']==P3)
    &(t_konta['P4']==P4)
],index=['Konto_N'],values=['Kwota_SAL'],columns=['M'],aggfunc=[np.sum],fill_value=0,margins=True,margins_name='RAZEM').reset_index()

t_tabela=pd.DataFrame(t_tabela.values.tolist(),columns=[P4,1,2,3,4,5,6,7,8,9,10,11,12,'RAZEM'])

t_tabela_fv=t_tabela.style.format({1:'{0:,.2f}',2:'{0:,.2f}',3:'{0:,.2f}',4:'{0:,.2f}',5:'{0:,.2f}',6:'{0:,.2f}',7:'{0:,.2f}',8:'{0:,.2f}',9:'{0:,.2f}',10:'{0:,.2f}',11:'{0:,.2f}',12:'{0:,.2f}','RAZEM':'{0:,.2f}'}).hide(axis='index')

t_tabela

,2. Produkty gotowe,1,2,3,4,5,6,7,8,9,10,11,12,RAZEM
0,000-000 - Zapis techniczny,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00
1,600-001 - WYROBY GOTOWE,"1,362,824.33","-55,491.72","118,409.98","-94,564.71","-352,593.72","106,127.66","250,142.90",0,0,0,0,0,"1,334,854.72"
2,RAZEM,"1,362,824.33","-55,491.72","118,409.98","-94,564.71","-352,593.72","106,127.66","250,142.90",0,0,0,0,0,"1,334,854.72"
